In [1]:
import pandas as pd
import numpy as np
import os
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db
from Port_Doctor import *
from Macro_Auto_Extractor import update_DB   #csv 덮어쓰기
from automate_sentense import automate_sentense

In [2]:
json = {'226340': 50, '002310': 40, '005930':20, '289220':30, '058470':5, '068270':6}

In [3]:
def end_to_end(json_file, term='Q', start='2003-01-01', end='2022-10-01'):
    project_id = 'portdoctor-test-356008'
    cred = credentials.Certificate("./portdoctor-test-356008-firebase-adminsdk-4007v-cdf4d6c85d.json")
    # https://console.firebase.google.com/project/portdoctor-test-356008/firestore/data/
    client = firebase_admin.initialize_app(cred, {'project_id': project_id})
    db = firestore.client()
    
    stocks = [x for x in json_file.keys()]
    holdings = [y for y in json_file.values()]
    items = os.listdir('./Data/price')
    newlist = []
    for names in items:
        if names.endswith(".csv"):
            newlist.append(names)
    data = pd.read_csv('./Data/price/'+newlist[-1], encoding='cp949')
    price = data[['종목코드', '종목명', '종가']]
    price.index = data['종목코드']
    price.drop('종목코드', axis=1, inplace=True)
    price = price.loc[[x for x in json_file.keys()]]
    price['수량'] = [y for y in json_file.values()]
    price['비중'] = price['종가']*price['수량']
    weight_sum = np.sum(price['비중'])
    weight= price['비중'].map(lambda x: x/weight_sum).tolist()
    
    loc = './'
#     update_DB(loc)
    output_json, home_macro = automate_sentense(loc, stocks, weight, term=term, start = start, end=end)
    
    # DB에 쓰기
    for i in [x for x in output_json.keys()]:
        locals()['doc_ref_{}'.format(i)] = db.collection('KY test').document('docu_{}'.format(i))
        locals()['doc_ref_{}'.format(i)].set(output_json[i])
#     for i in [x for x in output_json.keys()]:
#         locals()['doc_ref_{}'.format(i)] = db.collection('KY test').document('HANA')
#         locals()['doc_ref_{}'.format(i)].update(output_json[i])
    for i in [x for x in home_macro.keys()]:
        locals()['doc_ref_{}'.format(i)] = db.collection('Macro').document(i)
        locals()['doc_ref_{}'.format(i)].set(home_macro[i])
        
    print('DB updated')
    return output_json, home_macro

In [4]:
end_to_end(json, term='Q', start='2003-01-01', end='2022-10-01')

C:\Users\jky93\AppData\Local\Temp\ipykernel_1516\495836562.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price.drop('종목코드', axis=1, inplace=True)
C:\Users\jky93\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[0.015678810947978974, 0.26753670473083196, 0.22040964292187784, 0.15497553017944535, 0.12008337864781585, 0.22131593257205004]


  0%|          | 0/7 [00:00<?, ?it/s]

WTI   :   {'확대': ['226340', '002310', '005930', '058470'], '축소': []}
krwusd   :   {'확대': [], '축소': ['002310', '005930', '058470']}
Price_consumer   :   {'확대': [], '축소': ['058470']}
call_1d   :   {'확대': [], '축소': ['058470', '068270']}
Price_producer   :   {'확대': ['068270'], '축소': []}
spread1   :   {'확대': [], '축소': []}
KOSDAQ   :   {'확대': [], '축소': []}
spread2   :   {'확대': [], '축소': []}
news_sentiment   :   {'확대': [], '축소': []}
KOSPI   :   {'확대': [], '축소': []}
DB updated


({'금리': {'name': '금리',
   'title1': '금리민감도  -0.06523782290665493',
   'message1': '단기 금리와는 -6.524%, 장단기 스프레드와는 0.0% 민감도를 갖습니다. 포트닥터 이용자들은 평균적으로 0.5%의 민감도를 갖습니다. 떡상가즈아님의 포트폴리오는 금리변동에  반응하며 금리변동과  방향으로 움직이는 포트폴리오입니다. 이런 포트폴리오는 금리 이 전망될 때 좋은 수익률을 낼 가능성이 높습니다.',
   'title2': '금리 하락 전망',
   'message2': '단기금리가 6개월동안 상승세에 있습니다. 통안증권 91일물은 2.327%로 낮아지고 있으며 초단기금리인 콜금리 1일물는 2.212%입니다. 금리변동에 대한 시장의 기대를 나타내는 장단기 금리차(국고채 3년물 - 통안증권 91일물) 또한 1개월, 3개월 전에 비해 각각 -0.286%p, -0.7%p만큼 변동했습니다. 시장은 단기적으로 금리 상승세, 중기적으로 금리 하락세를 기대하고 있습니다.',
   'title3': '포트폴리오 내 종목별 영향',
   'message3': {'positive': [], 'negative': ['058470', '068270']}},
  '주가': {'name': '주가',
   'title1': '주가민감도  0.0',
   'message1': '주식시장과 통계적으로 유의하지 않은 포트폴리오입니다. 포트닥터 이용자들은 평균적으로 0.5%의 민감도를 갖습니다. 떡상가즈아님의 포트폴리오는 주식시장 변동으로부터 직접적인 영향이 매우 작은 포트폴리오입니다. 이런 포트폴리오는 주식시장 변동이 불확실하거나 확신이 없을 때 좋은 포지셔닝입니다.',
   'title2': '주가 상승 전망',
   'message2': '주가지수가 최근동안 상승세에 있습니다. KOSPI는 2492.651p이며 KOSDAQ는 823.933p를 기록 중입니다. 이는 1개월 전에 비해 각각 129.857p, 52.807p만큼 변동한